### University of California, Berkeley
### Master of Information and Data Science Program (MIDS)
### w205 - Fundamentals of Data Engineering

Year: 2024 <br>
Semester: Spring <br>
Section:006 <br>
Instructor: Doris Schioberg <br> <br>
Team Members: <br>
team member 1: Rini Gupta <br>
team member 2: Gia Nguyen <br>
team member 3: Bao Pham <br>
team member 4: Jaekwang Shin

# Investment Analytics: Neo4j Perspectives on NASDAQ-100 Stock Price Behavior (2011-2016)

## Notebook Structure

* Section 1: Importing Data
* Section 2: Data Preprocessing
* Section 3: Data Analysis
* Section 4: Similarity/Correlation Algorithm
    * Section 4.1: Pearson Correlation Coefficient with Exponential Weight
    * Section 4.2: Similarity Matrix
* Section 5: Neo4j Connection and Functions
    * Section 5.1: Neo4j - Graph Creation
    * Section 5.2: Neo4j - Louvain Modularity (with similarity matrix)
    * Section 5.3: Neo4j - PageRank (with similarity matrix)
    * Section 5.4: Neo4j - Betweenness (with similarity matrix)
    * Section 5.5: Neo4j - PageRank (with Pearson correlation coefficient)
    * Section 5.6: Neo4j - Betweenness (with Pearson correlation coefficient)

## Importing Libraries

In [1]:
import pandas as pd

In [2]:
import neo4j
import csv
import math
import numpy as np
import pandas as pd
import psycopg2
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from pandas.tseries.holiday import (AbstractHolidayCalendar, Holiday, USFederalHolidayCalendar, nearest_workday)
from pandas.tseries.offsets import CustomBusinessDay, WeekOfMonth
from dateutil.relativedelta import MO, TH
from itertools import combinations

## Section 1: Importing Data

In [3]:
stock_df = pd.read_csv("NASDAQ_100_Data_From_2010.csv", sep='\t')
stock_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Name
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.562591,493729600,AAPL
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.573935,601904800,AAPL
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.469369,552160000,AAPL
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.457407,477131200,AAPL
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.500339,447610800,AAPL


## Section 2: Data Preprocessing
We will first construct the pearson_df dataframe using the period_df dataframe for our Pearson correlation coefficient analysis. We will then use period_df to construct quarterly_df which allows us to compute our cosine similarity analysis.

In [4]:
stock_df.shape

(271680, 8)

In [5]:
stock_df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
Name          object
dtype: object

In [6]:
#change column "Date" to datetime format
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

#filter by period from year 2011 to end of 2016
#this is because the NASDAQ100 was consistent throughout these years with no major stock disruption events
period_df = stock_df[(stock_df['Date'].dt.year >= 2011) & (stock_df['Date'].dt.year <= 2016)]

In [7]:
period_df.shape

(134327, 8)

In [8]:
class NASDAQHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
        Holiday('MartinLutherKingJrDay', month=1, day=1, offset=pd.DateOffset(weekday=MO(3))),
        Holiday('PresidentsDay', month=2, day=1, offset=pd.DateOffset(weekday=MO(3))),
        Holiday('GoodFriday', year=2011, month=4, day=22),
        Holiday('GoodFriday', year=2012, month=4, day=6),
        Holiday('GoodFriday', year=2013, month=3, day=29),
        Holiday('GoodFriday', year=2014, month=4, day=18),
        Holiday('GoodFriday', year=2015, month=4, day=3),
        Holiday('GoodFriday', year=2016, month=3, day=25),
        Holiday('MemorialDay', month=5, day=31, offset=pd.DateOffset(weekday=MO(-1))),
        Holiday('IndependenceDay', month=7, day=4, observance=nearest_workday),
        Holiday('LaborDay', month=9, day=1, offset=pd.DateOffset(weekday=MO(1))),
        Holiday('ThanksgivingDay', month=11, day=1, offset=pd.DateOffset(weekday=TH(4))),
        Holiday('Christmas', month=12, day=25, observance=nearest_workday)
    ]

custom_calendar = CustomBusinessDay(calendar=NASDAQHolidayCalendar())

def business_days(start_year, end_year):
    business_days = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq=custom_calendar)
    return business_days

trading_days_2011_2016 = business_days(2011, 2016)

In [9]:
actual_trading_days = period_df.groupby('Name')['Date'].nunique()
hurricane_sandy_days = pd.to_datetime(['2012-10-29', '2012-10-30'])
adjusted_trading_days = trading_days_2011_2016.drop(hurricane_sandy_days)
max_trading_days = len(adjusted_trading_days)

missing_days = max_trading_days - actual_trading_days
# Create a summary DataFrame
summary_table = pd.DataFrame({
    'Ticker': missing_days.index,
    'Max Available Trading Days': max_trading_days,
    'Actual Trading Days': actual_trading_days.values,
    'Missing Trading Days': missing_days.values
}).set_index('Ticker')

summary_table

,Max Available Trading Days,Actual Trading Days,Missing Trading Days
Ticker,,,
AAPL,1510,1510,0
ADBE,1510,1510,0
ADI,1510,1510,0
ADP,1510,1510,0
ADSK,1510,1510,0
...,...,...,...
VRTX,1510,1510,0
WBA,1510,1510,0
WDAY,1510,1061,449


In [10]:
tickers_to_drop = summary_table[summary_table['Missing Trading Days'] > 0].index

# Filter out these tickers
period_df = period_df[~period_df['Name'].isin(tickers_to_drop)].reset_index()

In [11]:
period_df.head()

,index,Date,Open,High,Low,Close,Adj Close,Volume,Name
0,252,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.106220,445138400,AAPL
1,253,2011-01-04,11.872857,11.875000,11.719643,11.831786,10.158968,309080800,AAPL
2,254,2011-01-05,11.769643,11.940714,11.767857,11.928571,10.242065,255519600,AAPL
3,255,2011-01-06,11.954286,11.973214,11.889286,11.918929,10.233788,300428800,AAPL
4,256,2011-01-07,11.928214,12.012500,11.853571,12.004286,10.307076,311931200,AAPL


In [12]:
period_df.shape

(128350, 9)

#### Construct pearson_df

In [13]:
stock_sector = {
    'AAPL': 'Technology',
    'MNST': 'Consumer Staples',
    'NXPI': 'Technology',
    'NVDA': 'Technology',
    'NTES': 'Technology',
    'NFLX': 'Consumer Discretionary',
    'MU': 'Technology',
    'MTCH': 'Technology',
    'MSFT': 'Technology',
    'MRVL': 'Technology',
    'MELI': 'Consumer Discretionary',
    'INTC': 'Technology',
    'MDLZ': 'Consumer Staples',
    'MCHP': 'Technology',
    'MAR': 'Consumer Discretionary',
    'LULU': 'Consumer Discretionary',
    'LRCX': 'Technology',
    'KLAC': 'Technology',
    'KDP': 'Consumer Staples',
    'ISRG': 'Health Care',
    'ORLY': 'Consumer Discretionary',
    'PAYX': 'Consumer Discretionary',
    'PCAR': 'Consumer Discretionary',
    'PEP': 'Consumer Staples',
    'XEL': 'Utilities',
    'WBA': 'Consumer Staples',
    'VRTX': 'Health Care',
    'VRSN': 'Technology',
    'VRSK': 'Consumer Discretionary',
    'TXN': 'Technology',
    'TSLA': 'Consumer Discretionary',
    'TMUS': 'Telecommunications',
    'TCOM': 'Consumer Discretionary',
    'SWKS': 'Technology',
    'SNPS': 'Technology',
    'SIRI': 'Consumer Discretionary',
    'SGEN': 'Health Care', #dropped from NASDA in 2023
    'SBUX': 'Consumer Discretionary',
    'ROST': 'Consumer Discretionary',
    'REGN': 'Health Care',
    'QCOM': 'Technology',
    'INTU': 'Technology',
    'INCY': 'Health Care',
    'ADBE': 'Technology',
    'ANSS': 'Technology',
    'CERN': 'Health Care',
    'CDNS': 'Technology',
    'BKNG': 'Consumer Discretionary',
    'BIIB': 'Health Care',
    'BIDU': 'Technology',
    'AVGO': 'Technology',
    'ATVI': 'Telecommunications', #acquired by MSFT in 2023
    'ASML': 'Technology',
    'AMZN': 'Consumer Discretionary',
    'ILMN': 'Health Care',
    'AMGN': 'Health Care',
    'AMD': 'Technology',
    'AMAT': 'Technology',
    'ALGN': 'Health Care',
    'AEP': 'Utilities',
    'ADSK': 'Technology',
    'ADP': 'Technology',
    'ADI': 'Technology',
    'CHKP': 'Technology',
    'CHTR': 'Telecommunications',
    'CMCSA': 'Telecommunications',
    'COST': 'Consumer Discretionary',
    'IDXX': 'Health Care',
    'HON': 'Industrials',
    'GOOGL': 'Technology',
    'GOOG': 'Technology',
    'GILD': 'Health Care',
    'FISV': 'Consumer Discretionary', #changed name to FI in 2023
    'FAST': 'Consumer Discretionary',
    'EXC': 'Utilities',
    'EBAY': 'Consumer Discretionary',
    'EA': 'Technology',
    'DXCM': 'Health Care',
    'DLTR': 'Consumer Discretionary',
    'CTSH': 'Technology',
    'CTAS': 'Consumer Discretionary',
    'CSX': 'Industrials',
    'CSCO': 'Telecommunications',
    'CPRT': 'Consumer Discretionary',
    'XLNX': 'Technology', #acquired by AMD in 2022
}

In [14]:
pearson_df = period_df
pearson_df['Sector'] = pearson_df['Name'].map(stock_sector)
pearson_df.drop_duplicates(subset=['Name']).groupby('Sector')['Name'].count().sort_values(ascending=False)

Sector
Technology                35
Consumer Discretionary    22
Health Care               13
Consumer Staples           5
Telecommunications         5
Utilities                  3
Industrials                2
Name: Name, dtype: int64

In [15]:
pearson_df['Price Change ($)'] = pearson_df['Close'] - pearson_df['Open']
pearson_df['Price Change (%)'] = ((pearson_df['Close'] - pearson_df['Open']) / pearson_df['Open']) * 100

In [16]:
pearson_df['Price Change ($)'] = pearson_df['Price Change ($)'].round(2)
pearson_df['Price Change (%)'] = pearson_df['Price Change (%)'].round(2)

In [17]:
pearson_df.head()

,index,Date,Open,High,Low,Close,Adj Close,Volume,Name,Sector,Price Change ($),Price Change (%)
0,252,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.106220,445138400,AAPL,Technology,0.14,1.21
1,253,2011-01-04,11.872857,11.875000,11.719643,11.831786,10.158968,309080800,AAPL,Technology,-0.04,-0.35
2,254,2011-01-05,11.769643,11.940714,11.767857,11.928571,10.242065,255519600,AAPL,Technology,0.16,1.35
3,255,2011-01-06,11.954286,11.973214,11.889286,11.918929,10.233788,300428800,AAPL,Technology,-0.04,-0.30
4,256,2011-01-07,11.928214,12.012500,11.853571,12.004286,10.307076,311931200,AAPL,Technology,0.08,0.64


In [18]:
stock_2011_2016 = pearson_df[['Date', 'Price Change (%)', 'Name', 'Sector']]

In [19]:
scaler = StandardScaler()
for ticker in stock_2011_2016['Name'].unique():
    stock_2011_2016.loc[stock_2011_2016['Name'] == ticker, 'Price Change (%)'] = scaler.fit_transform(stock_2011_2016.loc[stock_2011_2016['Name'] == ticker, ['Price Change (%)']])

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [20]:
stock_2011_2016.head()

,Date,Price Change (%),Name,Sector
0,2011-01-03,0.932113,AAPL,Technology
1,2011-01-04,-0.248204,AAPL,Technology
2,2011-01-05,1.038039,AAPL,Technology
3,2011-01-06,-0.210374,AAPL,Technology
4,2011-01-07,0.500843,AAPL,Technology


#### Construct quarterly_df

In [21]:
#set index as date
period_df.set_index('Date', inplace=True)
period_df.head()

,index,Open,High,Low,Close,Adj Close,Volume,Name,Sector,Price Change ($),Price Change (%)
Date,,,,,,,,,,,
2011-01-03,252,11.630000,11.795000,11.601429,11.770357,10.106220,445138400,AAPL,Technology,0.14,1.21
2011-01-04,253,11.872857,11.875000,11.719643,11.831786,10.158968,309080800,AAPL,Technology,-0.04,-0.35
2011-01-05,254,11.769643,11.940714,11.767857,11.928571,10.242065,255519600,AAPL,Technology,0.16,1.35
2011-01-06,255,11.954286,11.973214,11.889286,11.918929,10.233788,300428800,AAPL,Technology,-0.04,-0.30
2011-01-07,256,11.928214,12.012500,11.853571,12.004286,10.307076,311931200,AAPL,Technology,0.08,0.64


In [22]:
#recategorize dates into quarters
#group by stock quarters, round values to 2 decimal
period_df.index.rename('Period', inplace=True)
quarterly_df = period_df.groupby([pd.Grouper(freq='Q'), 'Name']).agg({
    'Open': 'first',
    'Close': 'last',
    'Low': 'min',
    'High': 'max',
    'Adj Close': 'mean',
    'Volume': 'mean',
    'Name': 'first'
}).round(2)

#rename columns for clarity
quarterly_df = quarterly_df.rename(columns={
    'Open': 'Quarter Open',
    'Close': 'Quarter Close',
    'Low': 'Period Low',
    'High': 'Period High',
    'Adj Close': 'Avg Adj Close',
    'Volume': 'Avg Volume',
})

#adjust index name to YYYY Q# format for readability
quarterly_df.index = (quarterly_df.index.get_level_values(0).year).astype(str) + ' Q' + (quarterly_df.index.get_level_values(0).quarter).astype(str)
quarterly_df.head()

,Quarter Open,Quarter Close,Period Low,Period High,Avg Adj Close,Avg Volume,Name
Period,,,,,,,
2011 Q1,11.63,12.45,11.60,13.03,10.60,5.071604e+08,AAPL
2011 Q1,30.83,33.16,30.76,35.84,33.47,6.165548e+06,ADBE
2011 Q1,37.68,39.38,36.29,41.66,30.00,3.023402e+06,ADI
2011 Q1,41.04,45.05,41.03,45.22,33.56,2.851079e+06,ADP
2011 Q1,38.63,44.11,38.04,44.44,41.48,2.751219e+06,ADSK


In [23]:
#add columns for price change ($) and price change (%)
quarterly_df['Price Change ($)'] = quarterly_df['Quarter Close'] - quarterly_df['Quarter Open']
quarterly_df['Price Change (%)'] = ((quarterly_df['Quarter Close'] - quarterly_df['Quarter Open']) / quarterly_df['Quarter Open']) * 100

#round the price change to 2 decimal places
quarterly_df['Price Change ($)'] = quarterly_df['Price Change ($)'].round(2)
quarterly_df['Price Change (%)'] = quarterly_df['Price Change (%)'].round(2)

#reorganize column names for readability
quarterly_df = quarterly_df[['Name', 'Quarter Open', 'Quarter Close', 'Period Low', 'Period High', 'Avg Adj Close', 'Avg Volume', 'Price Change ($)', 'Price Change (%)']]
quarterly_df.head()

,Name,Quarter Open,Quarter Close,Period Low,Period High,Avg Adj Close,Avg Volume,Price Change ($),Price Change (%)
Period,,,,,,,,,
2011 Q1,AAPL,11.63,12.45,11.60,13.03,10.60,5.071604e+08,0.82,7.05
2011 Q1,ADBE,30.83,33.16,30.76,35.84,33.47,6.165548e+06,2.33,7.56
2011 Q1,ADI,37.68,39.38,36.29,41.66,30.00,3.023402e+06,1.70,4.51
2011 Q1,ADP,41.04,45.05,41.03,45.22,33.56,2.851079e+06,4.01,9.77
2011 Q1,ADSK,38.63,44.11,38.04,44.44,41.48,2.751219e+06,5.48,14.19


In [24]:
# Standardize for each stock to ensure that the standardization is done relative to each stock's distribution of prices
for ticker in quarterly_df['Name'].unique():
    quarterly_df.loc[quarterly_df['Name'] == ticker, 'Price Change (%)'] = scaler.fit_transform(quarterly_df.loc[quarterly_df['Name'] == ticker, ['Price Change (%)']])

## Section 3: Data Analysis

In [25]:
stock_2011_2016.shape

(128350, 4)

In [26]:
stock_2011_2016.isnull().sum()

Date                0
Price Change (%)    0
Name                0
Sector              0
dtype: int64

In [27]:
stock_2011_2016.describe()

,Price Change (%)
count,1.283500e+05
mean,7.906068e-19
std,1.000004e+00
min,-1.059849e+01
25%,-5.591138e-01
50%,-4.930291e-03
75%,5.610114e-01
max,9.390655e+00


In [28]:
#note that we went from 128350 rows to 2040 by categorizing into quarterly periods and limiting analysis from 2011-2016
quarterly_df.shape

(2040, 9)

In [29]:
#check number of nulls
quarterly_df.isnull().sum()

Name                0
Quarter Open        0
Quarter Close       0
Period Low          0
Period High         0
Avg Adj Close       0
Avg Volume          0
Price Change ($)    0
Price Change (%)    0
dtype: int64

In [30]:
stock_count= quarterly_df['Name'].value_counts() #note that some stocks are not in the NASDAQ100 every quarterly period
print(stock_count)

AAPL    24
MNST    24
NXPI    24
NVDA    24
NTES    24
        ..
CTAS    24
CSX     24
CSCO    24
CPRT    24
XLNX    24
Name: Name, Length: 85, dtype: int64


In [31]:
quarterly_df.describe()

,Quarter Open,Quarter Close,Period Low,Period High,Avg Adj Close,Avg Volume,Price Change ($),Price Change (%)
count,2040.000000,2040.000000,2040.000000,2040.000000,2040.000000,2.040000e+03,2040.000000,2.040000e+03
mean,79.305657,82.572828,72.319382,89.454368,77.094559,1.169531e+07,3.267172,-1.866699e-17
std,136.744657,142.488539,126.380753,154.362658,141.738488,4.147206e+07,18.510706,1.000245e+00
min,1.470000,1.510000,1.270000,1.880000,1.600000,2.378629e+05,-171.320000,-3.571444e+00
25%,24.922500,25.997500,22.670000,28.407500,21.715000,1.724864e+06,-1.372500,-6.857255e-01
50%,44.460000,46.325000,40.175000,49.395000,40.070000,3.128624e+06,1.515000,-3.889004e-02
75%,74.770000,78.365000,68.657500,83.575000,70.042500,6.975078e+06,5.405000,6.939351e-01
max,1463.830000,1471.490000,1422.190000,1600.930000,1493.180000,6.058437e+08,242.500000,3.844623e+00


In [32]:
quarterly_df.head()

,Name,Quarter Open,Quarter Close,Period Low,Period High,Avg Adj Close,Avg Volume,Price Change ($),Price Change (%)
Period,,,,,,,,,
2011 Q1,AAPL,11.63,12.45,11.60,13.03,10.60,5.071604e+08,0.82,0.165696
2011 Q1,ADBE,30.83,33.16,30.76,35.84,33.47,6.165548e+06,2.33,0.172975
2011 Q1,ADI,37.68,39.38,36.29,41.66,30.00,3.023402e+06,1.70,0.155514
2011 Q1,ADP,41.04,45.05,41.03,45.22,33.56,2.851079e+06,4.01,0.818336
2011 Q1,ADSK,38.63,44.11,38.04,44.44,41.48,2.751219e+06,5.48,0.523140


## Section 4: Similarity/Correlation Algorithm

### Section 4.1: Pearson's Correlation Coefficient with Exponential Weight

In [33]:
def weighted_pearson_corr(s1, s2, lambda_ = 0.94):
    n = len(s1)
    weights = np.array([lambda_ ** (n-1-i) for i in range(n)])
    weights /= np.sum(weights)
    
    # Calculate weighted means
    weighted_mean1 = np.average(s1, weights=weights)
    weighted_mean2 = np.average(s2, weights=weights)
    
    # Calculate weighted covariance and variances
    weighted_covariance = np.average((s1 - weighted_mean1) * (s2 - weighted_mean2), weights=weights)
    weighted_variance1 = np.average((s1 - weighted_mean1) ** 2, weights=weights)
    weighted_variance2 = np.average((s2 - weighted_mean2) ** 2, weights=weights)
    
    # Calculate weighted Pearson correlation
    weighted_pearson = weighted_covariance / np.sqrt(weighted_variance1 * weighted_variance2)
    return weighted_pearson

In [34]:
# only compare stocks from the same sector

def calculate_correlations_for_stock_pairs(data):
    stock_names = data['Name'].unique()
    correlation_results = []

    for stock1, stock2 in combinations(stock_names, 2):
        sector1 = data[data['Name'] == stock1]['Sector'].iloc[0]
        sector2 = data[data['Name'] == stock2]['Sector'].iloc[0]

        # Check if stocks are from the same sector
        if sector1 == sector2:
            s1_data = data[(data['Name'] == stock1) & (data['Sector'] == sector1)].set_index('Date')['Price Change (%)']
            s2_data = data[(data['Name'] == stock2) & (data['Sector'] == sector2)].set_index('Date')['Price Change (%)']

            if len(s1_data) == len(s2_data):
                correlation = weighted_pearson_corr(s1_data.values, s2_data.values)
                correlation_results.append({'stock1': stock1, 'stock2': stock2, 'sector': sector1, 'correlation': correlation})
            else:
                print(f"Mismatch found in trading days between {stock1} and {stock2}")

    return pd.DataFrame(correlation_results)

In [35]:
correlation_df = calculate_correlations_for_stock_pairs(stock_2011_2016)

In [36]:
correlation_df

,stock1,stock2,sector,correlation
0,AAPL,ADBE,Technology,0.575089
1,AAPL,ADI,Technology,0.487113
2,AAPL,ADP,Technology,0.494167
3,AAPL,ADSK,Technology,0.440092
4,AAPL,AMAT,Technology,0.520039
...,...,...,...,...
923,TCOM,VRSK,Consumer Discretionary,0.116902
924,TSLA,VRSK,Consumer Discretionary,0.366998
925,TXN,VRSN,Technology,0.225380
926,TXN,XLNX,Technology,0.570852


In [37]:
correlation_df.shape

(928, 4)

In [38]:
# We only want to highlight correlation that are at least -0.3 and under or 0.3 and up

f_corr_df = correlation_df[(correlation_df['correlation'] <= -0.3) | (correlation_df['correlation'] >= 0.3)]

In [39]:
f_corr_df.shape

(700, 4)

In [40]:
f_corr_df.head()

,stock1,stock2,sector,correlation
0,AAPL,ADBE,Technology,0.575089
1,AAPL,ADI,Technology,0.487113
2,AAPL,ADP,Technology,0.494167
3,AAPL,ADSK,Technology,0.440092
4,AAPL,AMAT,Technology,0.520039


In [41]:
f_corr_df.to_csv('filtered_corr_df.csv', index=True)

### Section 4.2: Cosine Similarity Matrix

In [42]:
#create copy
quarterly_df_1= quarterly_df
quarterly_df_1.reset_index(inplace=True)

price_change_percent_values = quarterly_df_1.pivot(index='Name', columns='Period', values='Price Change (%)')

#some stocks are not available for certain quarters, hence we fill that period as 0 instead of dropping the entire stock
price_change_percent_values = price_change_percent_values.fillna(0)

#perform cosine_similarity to get similarity matrix
similarity_matrix = cosine_similarity(price_change_percent_values)
print(similarity_matrix)

[[ 1.          0.12465694  0.26291418 ...  0.01856365 -0.26801875
   0.01701199]
 [ 0.12465694  1.          0.63709416 ...  0.45814783 -0.0577938
   0.66099339]
 [ 0.26291418  0.63709416  1.         ...  0.58477156  0.01397359
   0.67700198]
 ...
 [ 0.01856365  0.45814783  0.58477156 ...  1.          0.28713012
   0.37752785]
 [-0.26801875 -0.0577938   0.01397359 ...  0.28713012  1.
  -0.10042972]
 [ 0.01701199  0.66099339  0.67700198 ...  0.37752785 -0.10042972
   1.        ]]


In [54]:
#export to csv, uncomment to export

similarity_df = pd.DataFrame(similarity_matrix, index=price_change_percent_values.index, columns=price_change_percent_values.index)
similarity_df.head()

Name,AAPL,ADBE,ADI,ADP,ADSK,AEP,ALGN,AMAT,AMD,AMGN,...,TCOM,TMUS,TSLA,TXN,VRSK,VRSN,VRTX,WBA,XEL,XLNX
Name,,,,,,,,,,,,,,,,,,,,,
AAPL,1.000000,0.124657,0.262914,-0.033264,0.410524,-0.222930,0.176997,0.144261,0.025900,0.036626,...,-0.060064,0.106307,-0.077634,0.125529,0.104107,0.088148,0.022250,0.018564,-0.268019,0.017012
ADBE,0.124657,1.000000,0.637094,0.219216,0.763786,-0.039407,0.515931,0.607011,0.252533,0.346931,...,0.359882,0.260569,0.269017,0.765898,0.296113,0.243355,0.056712,0.458148,-0.057794,0.660993
ADI,0.262914,0.637094,1.000000,0.470291,0.591171,0.023633,0.423635,0.557714,0.230610,0.203596,...,0.085686,0.390175,0.020539,0.741868,0.340814,0.120538,-0.191563,0.584772,0.013974,0.677002
ADP,-0.033264,0.219216,0.470291,1.000000,0.295913,0.391271,0.521721,0.322479,0.206506,0.191756,...,-0.190535,0.345323,-0.039379,0.427111,0.271783,0.567984,0.024676,0.421720,0.395683,0.280659
ADSK,0.410524,0.763786,0.591171,0.295913,1.000000,-0.079660,0.397278,0.730501,0.368949,0.397314,...,0.228102,0.141502,-0.068098,0.794502,0.220760,0.303936,0.107751,0.353233,-0.124555,0.554642


In [44]:
similarity_df.to_csv('similarity_matrix.csv')

## Section 5: Neo4j Connection and Functions

In [45]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

In [46]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"

    query = "MATCH ()-[r]-() DELETE r"
    session.run(query)

    query = "MATCH (n) DELETE n"
    session.run(query)

In [47]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"

    result = session.run(query, **kwargs)

    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

    return df

In [48]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [49]:
def my_neo4j_create_node(name):
    "create a node with label name"

    query = """

    CREATE (:Stock {name: $name})

    """

    session.run(query, name=name)

In [50]:
def my_neo4j_create_node_sector(name, sector=None):
    "create a node with label name and optional sector"
    if sector:
        query = """
        CREATE (:Stock {name: $name, sector: $sector})
        """
        session.run(query, name=name, sector=sector)
    else:
        query = """
        CREATE (:Stock {name: $name})
        """
        session.run(query, name=name)

In [51]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

cursor = connection.cursor()

### Section 5.1: Neo4j - Graph Creation (Similarity Matrix)

In [52]:
my_neo4j_wipe_out_database()

In [55]:
#create nodes for each stock
for stock_name in similarity_df.index:
    my_neo4j_create_node(stock_name)

#create relationships between stocks based on the similarity matrix
for i, stock_name in enumerate(similarity_df.index):
    #sort similarity scores for the current stock
    sorted_scores = sorted(enumerate(similarity_df.iloc[i], start=0), key=lambda x: x[1], reverse=True)[:3]
    #create relationships with top three most similar stocks (excluding self-correlation)
    for j, (index, similarity_score) in enumerate(sorted_scores):
        if index != i:  #skip self-correlation
            similar_stock_name = similarity_df.index[index]
            #use similarity score as weight
            similarity_score = similarity_df.iloc[i, index]
            query = """
            MATCH (s1:Stock {name: $stock1}), (s2:Stock {name: $stock2})
            MERGE (s1)-[r:SIMILARITY]->(s2)
            SET r.score = $similarity_score
            """
            session.run(query, stock1=stock_name, stock2=similar_stock_name, similarity_score=similarity_score)
drop_query = """
CALL gds.graph.drop('similarity_graph', false)
"""
session.run(drop_query)

graph_creation_query = """

CALL gds.graph.project('similarity_graph', 'Stock', 'SIMILARITY', {relationshipProperties: ['score']})

"""
session.run(graph_creation_query)

### Section 5.2: Neo4j - Louvain Modularity (Similarity Matrix)

In [66]:
def neo4j_louvain_gds(df, session):

    drop_query = """
    CALL gds.graph.drop('similarity_graph', false)
    """
    session.run(drop_query)

    graph_creation_query = """

    CALL gds.graph.project('similarity_graph', 'Stock', 'SIMILARITY', {relationshipProperties: ['score']})

    """
    session.run(graph_creation_query)

    louvain_query = """
    CALL gds.louvain.stream('similarity_graph')
    YIELD nodeId, communityId, intermediateCommunityIds
    WITH nodeId, communityId, intermediateCommunityIds
    MATCH (s:Stock) WHERE id(s) = nodeId and s.name IS NOT NULL
    RETURN s.name AS stock, communityId AS community 
    ORDER BY communityId DESC
    """
    result_df = my_neo4j_run_query_pandas(louvain_query)

    community_assignment_query = """
    CALL gds.louvain.stream('similarity_graph', {includeIntermediateCommunities: true})
    YIELD nodeId, communityId, intermediateCommunityIds
    MATCH (s:Stock) WHERE id(s) = nodeId
    SET s.community_id = communityId
    LIMIT 85
    """
    my_neo4j_run_query_pandas(community_assignment_query)

    return result_df

In [61]:
louvain_df = neo4j_louvain_gds(similarity_df, session)
louvain_df = louvain_df.iloc[:len(louvain_df)//2]
louvain_df

,stock,community
0,AMGN,169
1,ASML,169
2,BIDU,169
3,DXCM,169
4,MELI,169
...,...,...
80,AMZN,98
81,ATVI,98
82,XEL,93
83,AEP,93


In [62]:
#print the stocks in each community
for community, group in louvain_df.groupby('community'):
    print(f"Community {community}:")
    for stock in group['stock']:
        print(stock)
    print()

Community 93:
XEL
AEP
EXC

Community 98:
AMZN
ATVI

Community 103:
EA
TCOM
TSLA

Community 104:
BIIB
EBAY
SGEN

Community 108:
WBA
AAPL
BKNG
CHKP
CTSH
INCY
LULU
ORLY
REGN

Community 112:
ALGN
CDNS
CMCSA
CPRT
CTAS
FAST
FISV
HON
INTU
SBUX

Community 126:
GOOG
GOOGL

Community 134:
COST
DLTR
KDP
ROST

Community 142:
CERN
ISRG
MNST

Community 149:
AMD
IDXX
INTC
MSFT
NVDA

Community 162:
XLNX
AVGO
CHTR
CSX
ILMN
MCHP
MRVL
MU
NXPI
PEP
QCOM
SWKS

Community 164:
SIRI
TMUS

Community 166:
ADBE
ADI
ADSK
AMAT
KLAC
LRCX
MAR
NFLX
PCAR
TXN

Community 168:
ADP
ANSS
CSCO
GILD
MDLZ
MTCH
PAYX
VRSK
VRSN

Community 169:
AMGN
ASML
BIDU
DXCM
MELI
NTES
SNPS
VRTX



In [67]:
community_stock_df = louvain_df.groupby('community')['stock'].apply(list).reset_index()

# Convert the list of stocks to a string if preferred
community_stock_df['stock'] = community_stock_df['stock'].apply(', '.join)

# Rename the columns for clarity
community_stock_df.columns = ['Community', 'Stocks']

In [68]:
community_stock_df

,Community,Stocks
0,93,"XEL, AEP, EXC"
1,98,"AMZN, ATVI"
2,103,"EA, TCOM, TSLA"
3,104,"BIIB, EBAY, SGEN"
4,108,"WBA, AAPL, BKNG, CHKP, CTSH, INCY, LULU, ORLY,..."
5,112,"ALGN, CDNS, CMCSA, CPRT, CTAS, FAST, FISV, HON..."
6,126,"GOOG, GOOGL"
7,134,"COST, DLTR, KDP, ROST"
8,142,"CERN, ISRG, MNST"
9,149,"AMD, IDXX, INTC, MSFT, NVDA"


### Section 5.3: Neo4j - PageRank (Similarity Matrix)

In [ ]:
def neo4j_pagerank_gds(df, session):

    drop_query = """
    CALL gds.graph.drop('similarity_graph', false)
    """
    session.run(drop_query)

    graph_creation_query = """

    CALL gds.graph.project('similarity_graph', 'Stock', 'SIMILARITY', {relationshipProperties: ['score']})

    """
    session.run(graph_creation_query)

    pagerank_query = """

    CALL gds.pageRank.stream('similarity_graph',
                             { maxIterations: $max_iterations,
                               dampingFactor: $damping_factor}
                             )
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score as page_rank
    ORDER BY page_rank DESC, name ASC

    """

    max_iterations = 20
    damping_factor = 0.05

    result_df= my_neo4j_run_query_pandas(pagerank_query, max_iterations=max_iterations, damping_factor=damping_factor)

    return result_df

In [ ]:
pagerank_df = neo4j_pagerank_gds(similarity_df, session)
pagerank_df

### Section 5.4: Neo4j - Betweenness (Similarity Matrix)

In [ ]:
def neo4j_betweenness_gds(df, session):

    drop_query = """
    CALL gds.graph.drop('similarity_graph', false)
    """
    session.run(drop_query)

    graph_creation_query = """

    CALL gds.graph.project('similarity_graph', 'Stock', 'SIMILARITY', {relationshipProperties: ['score']})

    """
    session.run(graph_creation_query)

    betweenness_query = """

    CALL gds.betweenness.stream('similarity_graph', {relationshipWeightProperty: 'score'})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
    ORDER BY betweenness DESC

    """
    result_df= my_neo4j_run_query_pandas(betweenness_query)

    return result_df

In [ ]:
betweenness_df = neo4j_betweenness_gds(similarity_df, session)
betweenness_df

### Section 5.5: Neo4j - PageRank (Pearson's Correlation)

In [ ]:
def neo4j_pagerank_gds(df, session):
    my_neo4j_wipe_out_database()
    
    for stock in pd.concat([df['stock1'], df['stock2']]).unique():
        my_neo4j_create_node_sector(stock)
    
    for index, row in df.iterrows():
        query = """
        MATCH (s1:Stock {name: $stock1}), (s2:Stock {name: $stock2})
        MERGE (s1)-[:CORRELATED_WITH {score: $correlation}]->(s2)
        """
        session.run(query, stock1=row['stock1'], stock2=row['stock2'], correlation=row['correlation'])

    drop_query = """
    CALL gds.graph.drop('pagerankGraph', false)
    """
    session.run(drop_query)
    
    graph_creation_query = """
    CALL gds.graph.project(
        'pagerankGraph',
        'Stock',
        {
            CORRELATED_WITH: {
                type: 'CORRELATED_WITH',
                properties: 'score'
            }
        }
    )
    """
    session.run(graph_creation_query)

    pagerank_query = """
    CALL gds.pageRank.stream('pagerankGraph', {relationshipWeightProperty: 'score'})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS stock, score AS pagerank
    ORDER BY pagerank DESC
    """
    result_df = my_neo4j_run_query_pandas(pagerank_query)
    
    return result_df

In [ ]:
pr_df = neo4j_pagerank_gds(f_corr_df, session)
pr_df

### Section 5.6: Neo4j -Betweenness (Pearson's Correlation)

In [ ]:
pd.options.display.max_rows = None

In [ ]:
def neo4j_betweenness_centrality_gds(df, session):
    my_neo4j_wipe_out_database()
    
    for stock in pd.concat([df['stock1'], df['stock2']]).unique():
        my_neo4j_create_node_sector(stock)
    
    for index, row in df.iterrows():
        query = """
        MATCH (s1:Stock {name: $stock1}), (s2:Stock {name: $stock2})
        MERGE (s1)-[:CORRELATED_WITH {score: $correlation}]->(s2)
        """
        session.run(query, stock1=row['stock1'], stock2=row['stock2'], correlation=row['correlation'])

    drop_query = """
    CALL gds.graph.drop('betweennessGraph', false)
    """
    session.run(drop_query)
    
    graph_creation_query = """
    CALL gds.graph.project(
        'betweennessGraph',
        'Stock',
        {
            CORRELATED_WITH: {
                type: 'CORRELATED_WITH',
                properties: 'score'
            }
        }
    )
    """
    session.run(graph_creation_query)

    betweenness_centrality_query = """
    CALL gds.betweenness.stream('betweennessGraph', {relationshipWeightProperty: 'score'})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS stock, score AS betweenness
    ORDER BY betweenness DESC
    """
    result_df = my_neo4j_run_query_pandas(betweenness_centrality_query)
    
    return result_df

In [ ]:
result_df = neo4j_betweenness_centrality_gds(f_corr_df, session)
result_df